# OLR outgoing longwave radiation

Information about olr

https://www.ncdc.noaa.gov/teleconnections/enso/indicators/olr/

https://en.wikipedia.org/wiki/Outgoing_longwave_radiation

**This notebook will extract the olr data from the NetCDF file**

I used panoply to get an understanding of the structure of the data when I was planning how to extract the data.  I understand that there are libraries available in other languages but I want to learn what is available in the Julia ecosystem.

In [4]:
using CSV
using DataFrames
using Dates
using JuliaDB
using NetCDF

In [39]:
function extract_olr(day)
    depth = []
    for i in range(1, stop=180, step=1)
        for j in range(1, stop=360, step=1)
            push!(depth, day[j,i])
        end 
    end
    return depth
end

extract_olr (generic function with 1 method)

In [7]:
# used to create a latitude
function create_latitude(start, stop, step)
#    print("Inside create_latitude function \n")
    coordinates = []
    for i in range(start, stop=stop, step=step)
        for j in 1:360
            push!(coordinates,i)
        end
    end
    return coordinates # Julia automatically returns the last statement
end

create_latitude (generic function with 1 method)

In [22]:
# Longitude
function create_longitude()
    return collect(Iterators.flatten([collect(-180:1:179) for i in 1:180]))
end

create_longitude (generic function with 1 method)

## OLR NetCDF data files will be ETL.
Extracted from NetCDF, transormed to and array that is then written as a CSV.

**File name will be yyyy_day.csv**

In [40]:
function create_daily_olr(latitude, longitude, csv_file, daily_olr)
    olr=extract_olr(daily_olr)
    data = hcat(latitude, longitude, olr)
    filename = string("../olr_csv/", csv_file)
    if !(isfile(filename))
        CSV.write(filename, DataFrame(data), header=false)
    end
end

create_daily_olr (generic function with 1 method)

In [36]:
function create_yearly_olr(latitude, longitude, file_year, olr_data)
    days_in_year = length(olr_data[1,1,:])
    for day in range(1,stop=days_in_year, step=1)
        csv_file = string(file_year, "_", day, ".csv")
        create_daily_olr(latitude, longitude, csv_file, olr_data[:,:,day])
    end
end

create_yearly_olr (generic function with 1 method)

In [37]:
function transform_olr_data()
    olr_files = readdir("../olr_data/")
    latitude = create_latitude(-90, 89, 1)
    longitude = create_longitude()
    
    for file in olr_files
        file_year = file[18:21]
        yearly_olr_data = NetCDF.open(string("../olr_data/", file), "olr")
        create_yearly_olr(latitude, longitude, file_year, yearly_olr_data)
    end
end

transform_olr_data (generic function with 1 method)

In [38]:
@time transform_olr_data()


 17.295986 seconds (497.59 M allocations: 27.884 GiB, 11.60% gc time)
